# Assemble microbial genomes

Let's assemble the microbial genomes so we can measure accuracy of HowDeSBT/BIGSI when compared to alignment based methods (e.g., BLAST).

First, let's setup some environment variables.

In [48]:
data_type=microbial
assembly_dir=assembly
data_dir=data-downsampled
queries_dir=queries

threads=4
jobs=12

PROJECT_DIR=`git rev-parse --show-toplevel`
cd $PROJECT_DIR

The code given below assumes you have the following [conda](https://docs.conda.io/en/latest/) environments setup to install [skesa](https://github.com/ncbi/SKESA) and [staramr](https://github.com/phac-nml/staramr). This can be done with.

```bash
conda create --name staramr staramr
conda create --name skesa skesa
```

Let's verify these commands exist (and verify versions).

In [13]:
conda run --name skesa skesa --version
conda run --name staramr staramr --version

SKESA v.2.3.0
skesa --version 

staramr 0.5.1


Great. Let's now work at assembling all the genomes.

## Assemble data

For a fair comparison to BIGSI/HowDeSBT, we will only assemble the downsampled data (treating the paired-end datasets as single end). Let's do this now.

In [16]:
input_dir=${data_type}/${data_dir}
output_dir=${data_type}/${assembly_dir}

mkdir ${output_dir}

before=`date +%s`

commands_file=`mktemp`
for file in ${input_dir}/*.fastq.gz
do
    accession=`basename ${file} .fastq.gz`

    output_file=${output_dir}/${accession}.fasta
    output_log=${output_dir}/skesa.${accession}.log

    command="/usr/bin/time -v skesa --cores ${threads} --fastq ${file} --contigs_out ${output_file} \
        1> ${output_log} 2> ${output_log}.err"
    echo ${command} >> ${commands_file}
done

command="parallel -j ${jobs} -a ${commands_file}"
echo "Will run commands from [${commands_file}] like:"
head -n 1 ${commands_file}

echo ${command}
conda run --name skesa ${command}

after=`date +%s`
minutes=`echo "(${after}-${before})/60" | bc -l`
printf "Done. Took %0.2f minutes.\n" ${minutes}

mkdir: cannot create directory ‘microbial/assembly’: File exists
Will run commands from [/tmp/tmp.kIhvDNvSaj] like:
/usr/bin/time -v skesa --cores 4 --fastq microbial/data-downsampled/ERR1144974.fastq.gz --contigs_out microbial/assembly/ERR1144974.fasta 1> microbial/assembly/skesa.ERR1144974.log 2> microbial/assembly/skesa.ERR1144974.log.err
parallel -j 12 -a /tmp/tmp.kIhvDNvSaj
Done. Took 0.62 minutes.


Finished. So let's look at the results.

In [18]:
ls -lh ${data_type}/${assembly_dir}/*.fasta | head

-rw-r--r-- 1 apetkau grp_apetkau  175K Dec 12 11:38 microbial/assembly/ERR1144974.fasta
-rw-r--r-- 1 apetkau grp_apetkau  198K Dec 12 11:38 microbial/assembly/ERR1144975.fasta
-rw-r--r-- 1 apetkau grp_apetkau  198K Dec 12 11:38 microbial/assembly/ERR1144976.fasta
-rw-r--r-- 1 apetkau grp_apetkau  184K Dec 12 11:38 microbial/assembly/ERR1144977.fasta
-rw-r--r-- 1 apetkau grp_apetkau  201K Dec 12 11:38 microbial/assembly/ERR1144978.fasta
-rw-r--r-- 1 apetkau grp_apetkau  274K Dec 12 11:38 microbial/assembly/ERR3655992.fasta
-rw-r--r-- 1 apetkau grp_apetkau  223K Dec 12 11:38 microbial/assembly/ERR3655994.fasta
-rw-r--r-- 1 apetkau grp_apetkau  261K Dec 12 11:38 microbial/assembly/ERR3655996.fasta
-rw-r--r-- 1 apetkau grp_apetkau  265K Dec 12 11:38 microbial/assembly/ERR3655998.fasta
-rw-r--r-- 1 apetkau grp_apetkau  247K Dec 12 11:38 microbial/assembly/ERR3656002.fasta


These are all pretty small files (much less than what is expected for a full microbial genome which should be on the order of multi MB as they are on the order of multiple base pairs in length). But, this is expected as we used dramatically reduced reads as input (max 10 million bp per read set).

Let's now search these genomes using `staramr` (which uses BLAST) for specific antimicrobial resistance genes and plasmind incompatibility factors which we will attempt to recover using BIGSI/HowDeSBT.

## Search for AMR genes with StarAMR

In [19]:
staramr_out_dir=${data_type}/${assembly_dir}/staramr
staramr_in_dir=${data_type}/${assembly_dir}
conda run --name staramr staramr search -o ${staramr_out_dir} ${staramr_in_dir}/*.fasta

2019-12-12 11:45:02,734 INFO: No --pointfinder-organism specified. Will not search the PointFinder databases
2019-12-12 11:45:02,735 INFO: No --plasmidfinder-database-type specified. Will search the entire PlasmidFinder database
2019-12-12 11:45:02,735 INFO: --output-dir set. All files will be output to [microbial/assembly/staramr]
2019-12-12 11:45:02,735 INFO: Will exclude ResFinder/PointFinder genes listed in [/home/CSCScience.ca/apetkau/miniconda3/envs/staramr/lib/python3.7/site-packages/staramr/databases/exclude/data/genes_to_exclude.tsv]. Use --no-exclude-genes to disable
2019-12-12 11:45:03,136 INFO: Making BLAST databases for input files
2019-12-12 11:45:03,539 INFO: Scheduling blasts for ERR1144974.fasta
2019-12-12 11:45:03,644 INFO: Scheduling blasts for ERR1144975.fasta
2019-12-12 11:45:03,780 INFO: Scheduling blasts for ERR1144976.fasta
2019-12-12 11:45:03,938 INFO: Scheduling blasts for ERR1144977.fasta
2019-12-12 11:45:04,063 INFO: Scheduling blasts for ERR1144978.fasta
20

Awesome. Finished. Let's take a look at what was found.

In [22]:
cat ${staramr_out_dir}/summary.tsv

Isolate ID	Genotype	Predicted Phenotype	Plasmid
ERR1144974	None	Sensitive	None
ERR1144975	None	Sensitive	None
ERR1144976	None	Sensitive	None
ERR1144977	None	Sensitive	None
ERR1144978	None	Sensitive	None
ERR3655992	None	Sensitive	None
ERR3655994	None	Sensitive	None
ERR3655996	None	Sensitive	None
ERR3655998	None	Sensitive	None
ERR3656002	None	Sensitive	None
ERR3656004	None	Sensitive	None
ERR3656010	None	Sensitive	None
ERR3656012	None	Sensitive	None
ERR3656013	None	Sensitive	None
ERR3656015	None	Sensitive	None
ERR3656018	aac(6')-Ib-cr, aac(6')-Ib-cr, dfrA27, sul1	gentamicin, ciprofloxacin I/R, trimethoprim, sulfisoxazole	IncFIA(HI1)
ERR3656019	None	Sensitive	None
SRR10298903	None	Sensitive	None
SRR10298904	None	Sensitive	None
SRR10298905	None	Sensitive	None
SRR10298906	None	Sensitive	None
SRR10298907	None	Sensitive	None
SRR10512964	None	Sensitive	None
SRR10512965	aac(6')-Ib-cr, aac(6')-Ib-cr, sul1	gentamicin, ciprofloxacin I/R, sulfisoxazole	IncN
SRR10512968	blaSHV-31, sul1	ampicillin, ce

This shows a summary of the detected antimicrobial resistance genes as well as plasmind incompatibilty factors. Most are listed as finding **None** for genes as we are working with only fractions of a genome.

In [33]:
cat ${staramr_out_dir}/plasmidfinder.tsv

Isolate ID	Plasmid	%Identity	%Overlap	HSP Length/Total Length	Contig	Start	End	Accession
ERR3656018	IncFIA(HI1)	98.47	67.27	261/388	Contig_468_3.70863	38	298	AF250878
SRR10512965	IncN	99.81	100.00	514/514	Contig_1087_3.83191	113	626	AY046276
SRR10527348	Col(BS512)	100.00	100.00	233/233	Contig_1_18.8607_Circ	632	400	010656
SRR10527349	IncFII	100.00	89.66	234/261	Contig_645_4.83077	345	112	AY458016
SRR10527351	Col(BS512)	100.00	100.00	233/233	Contig_512_9.2097	918	686	010656
SRR10527351	IncI1	100.00	78.87	112/142	Contig_457_4.26061	1	112	AP005147
SRR10527352	Col(BS512)	100.00	100.00	233/233	Contig_464_16.3518	918	686	010656


This shows the plasmind incompatibility factors detected in the genome. We are specifically looking for something with **%Identity** at 100 and **%Overlap** at 100 so we can use exact matching for our searches.

In [40]:
grep -P '100.00\t100.00' ${staramr_out_dir}/plasmidfinder.tsv

SRR10527348	Col(BS512)	100.00	100.00	233/233	Contig_1_18.8607_Circ	632	400	010656
SRR10527351	Col(BS512)	100.00	100.00	233/233	Contig_512_9.2097	918	686	010656
SRR10527352	Col(BS512)	100.00	100.00	233/233	Contig_464_16.3518	918	686	010656


Okay. So it looks like this exact same sequence (of length `233`) is found in 3 genomes. We can use this as the **true** matches for comparison to HowDeSBT and BIGSI.

Let's pull out the genomic sequence (based on the accession number `010656`) and place into our `queries` directory.

In [51]:
grep -A 5 '010656' ${staramr_out_dir}/hits/plasmidfinder_SRR10527348.fasta |
    tee ${queries_dir}/accuracy_query.fasta

>Col(BS512)_1__NC_010656 isolate: SRR10527348, contig: Contig_1_18.8607_Circ, contig_start: 632, contig_end: 400, database_gene_start: 1, database_gene_end: 233, hsp/length: 233/233, pid: 100.00%, plength: 100.00%
ATGAATGCGGCGTTTAAGCGAATGGAAAAGCGAAAGGAGCTATCACCTGTTCAGGGGTGG
ATCAGGGCTACGGAGGTGACGCGAGGTAAGGATGGCAGCGCACATCCGCATTTTCACTGT
CTGCTGATGGTGCAACCTTCTTGGTTTAAAGGGAAGAACTACGTTAAGCACGAACGTTGG
GTAGAACTCTGGCGCGATTGCTTGCGGGTGAACTATGAGCCGAATATCGATAT


Awesome. We've now extracted a gene which we know exists in the assemblies for our accuracy comparisons.